#### ONNX Test Conversion and ONNXRuntime Test Run

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transform
import matplotlib.pyplot as plt
import numpy as np
from torchsummary import summary
import time
import torch.nn.functional as F
import os

In [ ]:
channel_size, image_size, image_size = 3, 224, 224
batch_size = 16
num_classes = 31

In [ ]:
!wget "https://drive.usercontent.google.com/download?id=10f3jNWltQyham_ebppXWfmyMftcFm8KS&export=download&authuser=0&confirm=t&uuid=bb96f63b-7200-4e05-b55d-7fb5753b7408&at=AN_67v2PXWp4HAHmqfH7S3e5wH-w:1728131372261" -O "Office-31.zip"

--2024-10-09 17:29:25--  https://drive.usercontent.google.com/download?id=10f3jNWltQyham_ebppXWfmyMftcFm8KS&export=download&authuser=0&confirm=t&uuid=bb96f63b-7200-4e05-b55d-7fb5753b7408&at=AN_67v2PXWp4HAHmqfH7S3e5wH-w:1728131372261
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.99.132, 2607:f8b0:400e:c0c::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.250.99.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79538472 (76M) [application/octet-stream]
Saving to: ‘Office-31.zip’

Office-31.zip       100%[===================>]  75.85M  49.6MB/s    in 1.5s    

2024-10-09 17:29:29 (49.6 MB/s) - ‘Office-31.zip’ saved [79538472/79538472]



In [ ]:
!unzip "Office-31.zip"

Archive:  Office-31.zip
  inflating: Office-31/amazon/back_pack/frame_0001.jpg  
  inflating: Office-31/amazon/back_pack/frame_0002.jpg  
  inflating: Office-31/amazon/back_pack/frame_0003.jpg  
  inflating: Office-31/amazon/back_pack/frame_0004.jpg  
  inflating: Office-31/amazon/back_pack/frame_0005.jpg  
  inflating: Office-31/amazon/back_pack/frame_0006.jpg  
  inflating: Office-31/amazon/back_pack/frame_0007.jpg  
  inflating: Office-31/amazon/back_pack/frame_0008.jpg  
  inflating: Office-31/amazon/back_pack/frame_0009.jpg  
  inflating: Office-31/amazon/back_pack/frame_0010.jpg  
  inflating: Office-31/amazon/back_pack/frame_0011.jpg  
  inflating: Office-31/amazon/back_pack/frame_0012.jpg  
  inflating: Office-31/amazon/back_pack/frame_0013.jpg  
  inflating: Office-31/amazon/back_pack/frame_0014.jpg  
  inflating: Office-31/amazon/back_pack/frame_0015.jpg  
  inflating: Office-31/amazon/back_pack/frame_0016.jpg  
  inflating: Office-31/amazon/back_pack/frame_0017.jpg  
  infla

In [ ]:
device = "cpu"
print(device)

cpu


In [ ]:
import torchvision.transforms as transforms
transformer = transforms.Compose([
    torchvision.models.MobileNet_V3_Small_Weights.IMAGENET1K_V1.transforms(),
])

In [ ]:
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split, ConcatDataset

DATASET_DIR = '/content/Office-31'

amazon_dataset = datasets.ImageFolder(root=f'{DATASET_DIR}/amazon', transform=transformer)
dslr_dataset = datasets.ImageFolder(root=f'{DATASET_DIR}/dslr', transform=transformer)
webcam_dataset = datasets.ImageFolder(root=f'{DATASET_DIR}/webcam', transform=transformer)

classes = webcam_dataset.classes
labeled_dataset = ConcatDataset([amazon_dataset, webcam_dataset])

In [ ]:
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split, ConcatDataset

Ds_amazon, Ds_amazon_test = random_split(amazon_dataset, [0.8, 0.2])
Ds_dslr, Ds_dslr_test = random_split(dslr_dataset, [0.8, 0.2])
Ds_webcam, Ds_webcam_test = random_split(webcam_dataset, [0.8, 0.2])

Dl_labeled = DataLoader(labeled_dataset, batch_size, shuffle=True, num_workers=2)

In [ ]:
path = '/content/cp/best_model.pth'
my_model = torch.load(path, weights_only=False, map_location=device)

In [ ]:
!pip install onnx
!pip install onnxscript
!pip install onnxruntime-gpu
import onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 100.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.7/670.7 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.2/226.2 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.8 MB/s eta 0:00:00


In [ ]:

base_onnx = '/content/cp/onnx_fp32.onnx'
torch_model = my_model.to(device)
input_shape = (1, 3, 224, 224)
torch_input = torch.randn(input_shape).to(device)
input_names = ['input']
output_names = ['output']
torch.onnx.export(torch_model, torch_input, base_onnx)


# Import library and load from ONNX side

import onnx
onnx_model = onnx.load(base_onnx)
onnx.checker.check_model(onnx_model)



In [ ]:
import time
num_iterations = 10000

dtypes = torch.float32
pytorch_fps = 0

with torch.no_grad():
    total_time = 0.0

    model = my_model
    model.to(device, dtype=torch.float32)

    for _ in range(num_iterations):
        input_data = torch.randn(input_shape, dtype=torch.float32).cpu()

        start_time = time.time()

        output_data = model(input_data)

        end_time = time.time()

        total_time += end_time - start_time

    pytorch_fps = num_iterations / total_time

    print(f"dtype: {dtypes}, PyTorch FPS: {pytorch_fps:.2f}")

dtype: torch.float32, PyTorch FPS: 116.23


In [ ]:
print(f"Estimate inference time:", (1/pytorch_fps)*1000 , "ms")

Estimate inference time: 8.603763389587403 ms


In [ ]:
def val_with_img(model, astype = np.float32):
  correct = 0
  total = 0

  dslr_dataset = datasets.ImageFolder(root=f'{DATASET_DIR}/dslr', transform=transformer)
  Dl_dslr = DataLoader(dslr_dataset, 1, shuffle=False)

  with torch.no_grad():
      for data in Dl_dslr:
          images, labels = data

          outputs = model.run(images.cpu().numpy().astype(astype))[0]

          correct += (outputs.argmax(1) == labels).float().sum().item()
          total += labels.shape[0]


  print('Accuracy : %f %%' % (100 * correct / total))

In [ ]:
import os

os.environ["ALLOW_RELEASED_ONNX_OPSET_ONLY"] = "0"
import onnxruntime.backend as backend

In [ ]:
import onnx
import onnxruntime
import torch
model_onnx = onnx.load('/content/cp/onnx_fp32.onnx')
onnx_engine = backend.prepare(model_onnx, provider='CPUExecutionProvider')

num_iterations = 10000
total_time_fp32 = 0.0
print("start inferencing...FP32")
with torch.no_grad():
    for i in range(num_iterations):
        input_data = torch.randn(input_shape, dtype=torch.float32).cpu()
        start_time = time.time()

        output_data = onnx_engine.run(input_data.cpu().numpy())[0]

        end_time = time.time()
        total_time_fp32 += end_time - start_time
tensor_fps_fp32 = num_iterations /total_time_fp32
#tensor_fps = num_iterations / total_time
print(f"Tensor FPS [FP32]: {tensor_fps_fp32:.2f}")
print(f"Speedup: {tensor_fps_fp32/pytorch_fps:.2f}x")

*************** EP Error ***************
EP Error /onnxruntime_src/onnxruntime/python/onnxruntime_pybind_state.cc:490 void onnxruntime::python::RegisterTensorRTPluginsAsCustomOps(PySessionOptions&, const onnxruntime::ProviderOptions&) Please install TensorRT libraries as mentioned in the GPU requirements page, make sure they're in the PATH or LD_LIBRARY_PATH, and that your GPU is supported.
 when using ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
Falling back to ['CUDAExecutionProvider', 'CPUExecutionProvider'] and retrying.
****************************************
start inferencing...FP32
Tensor FPS [FP32]: 400.69
Speedup: 3.45x


In [ ]:
#---------Inference with FP32--------------
import onnxruntime as ort
input_data_for_inference_fp32 = torch.randn(input_shape, dtype=torch.float32)
output_data = onnx_engine.run(input_data_for_inference_fp32.cpu().numpy())[0]

print(f"\n\nONNXRuntime [FP32] Output length: {len(output_data)}")
print(f"ONNXRuntime [FP32] Sample output: {output_data}")
print(f"ONNXRuntime [FP32] Sample output type: {output_data.dtype}")
print(f"Estimate inference time:", (1/tensor_fps_fp32)*1000 , "ms")



ONNXRuntime [FP32] Output length: 1
ONNXRuntime [FP32] Sample output: [[ -3.6665113    3.2595882   -8.609616     4.2981663  -11.035136
   -2.2603228   -9.905256    -2.464545    -5.2127337    5.0002975
  -15.275978     0.5990612   -6.8902044    3.589899   -10.123336
  -14.735673    -7.3838806   -5.136868     3.5077813    8.091756
    3.8092613   -7.483626    -6.515829    -6.5424514   -0.98734814
    8.470727     5.027675   -17.555645    -9.530277    -6.1149497
    9.176316  ]]
ONNXRuntime [FP32] Sample output type: float32
Estimate inference time: 2.7413158655166625 ms


In [ ]:
val_with_img(onnx_engine)

Accuracy : 96.787149 %


In [ ]:
# @title Optimized FP16 (Half Tensor)
print("Optimized model FP16 bench testing...")

torch_model_fp16 = my_model.cpu().half()
dummy_input_fp16 = torch.randn(input_shape, dtype=torch.float16).cpu().half()
input_names = ['input']
output_names = ['output']
torch.onnx.export(torch_model_fp16, dummy_input_fp16, '/content/cp/onnx_fp16.onnx', verbose=False, input_names=input_names, output_names=output_names)

# Create a engine from the ONNX model and measure inference speed
model_onnx_fp16 = onnx.load('/content/cp/onnx_fp16.onnx')
onnx_engine_fp16 = backend.prepare(model_onnx_fp16, provider='CPUExecutionProvider', float16=True)

num_iterations = 10000
total_time_fp16 = 0.0
print("start inferencing...FP16")
with torch.no_grad():
    for i in range(num_iterations):
        input_data = torch.randn(input_shape).cpu().numpy().astype(np.float16)

        start_time = time.time()

        output_data = onnx_engine_fp16.run(input_data)[0]

        end_time = time.time()

        total_time_fp16 += end_time - start_time

tensor_fps_fp16 = num_iterations /total_time_fp16
#tensor_fps = num_iterations / total_time
print(f"Tensor FPS [FP16]: {tensor_fps_fp16:.2f}")
print(f"Speedup: {tensor_fps_fp16/pytorch_fps:.2f}x")





#---------Inference with FP16--------------
import onnxruntime as ort
input_data_for_inference_fp16 = torch.randn(input_shape, dtype=torch.float16)
output_data = onnx_engine_fp16.run(input_data_for_inference_fp16.cpu().numpy())[0]

print(f"\n\nONNXRuntime [FP16] Output length: {len(output_data)}")
print(f"ONNXRuntime [FP16] Sample output: {output_data}")
print(f"ONNXRuntime [FP16] Sample output type: {output_data.dtype}")

Optimized model FP16 bench testing...
*************** EP Error ***************
EP Error /onnxruntime_src/onnxruntime/python/onnxruntime_pybind_state.cc:490 void onnxruntime::python::RegisterTensorRTPluginsAsCustomOps(PySessionOptions&, const onnxruntime::ProviderOptions&) Please install TensorRT libraries as mentioned in the GPU requirements page, make sure they're in the PATH or LD_LIBRARY_PATH, and that your GPU is supported.
 when using ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
Falling back to ['CUDAExecutionProvider', 'CPUExecutionProvider'] and retrying.
****************************************
start inferencing...FP16
Tensor FPS [FP16]: 414.13
Speedup: 3.56x


ONNXRuntime [FP16] Output length: 1
ONNXRuntime [FP16] Sample output: [[ -3.26   -7.953  -7.723   3.52  -14.87    3.43  -10.28   -5.723   1.868
    0.868 -13.734  -0.573  -6.367  -0.243  -1.285 -15.48   -6.203  -6.793
   -3.252   6.008   8.46   -6.53   -3.75   -2.906   1.308   2.174   4.

In [ ]:
print(f"Estimate inference time:", (1/tensor_fps_fp16)*1000 , "ms")

Estimate inference time: 2.414681506156921 ms


In [ ]:
val_with_img(onnx_engine_fp16, np.float16)

Accuracy : 96.586345 %


In [ ]:
# @title Optimized and Compressed (Quantization) to support INT8
import torch
import torch.quantization

import onnx
from onnxruntime.quantization import quantize_dynamic, QuantType

model_int8 = quantize_dynamic(
    '/content/cp/onnx_fp32.onnx',
    '/content/cp/onnx_pdqt_int8.onnx',
    weight_type=QuantType.QUInt8  # You can use QuantType.QUInt8 for unsigned INT8
)
#---------------------------------------------------------


# Create a engine from the ONNX model and measure inference speed
model_onnx_int8_pdqt = onnx.load('/content/cp/onnx_pdqt_int8.onnx')
onnx_engine_int8_pdqt = backend.prepare(model_onnx_int8_pdqt, provider='CPUExecutionProvider', float16=True)

# Run a calibration step to collect statistics
num_iterations = 10000
total_time_int8 = 0.0
with torch.no_grad():
    for i in range(num_iterations):
        input_data = torch.randn(input_shape, dtype=torch.float32).cpu()
        start_time = time.time()
        output_data = onnx_engine_int8_pdqt.run(input_data.cpu().numpy())[0]
        end_time = time.time()
        total_time_int8 += end_time - start_time
tensor_fps_int8 = num_iterations /total_time_int8
#tensor_fps = num_iterations / total_time

print(f"Tensor FPS [INT8]: {tensor_fps_int8:.2f}")
print(f"Speedup: {tensor_fps_int8/pytorch_fps:.2f}x")

#---------Inference with INT8--------------
import onnxruntime as ort
input_data_for_inference = torch.randn(input_shape, dtype=torch.float32)
output_quan_oxrun = onnx_engine_int8_pdqt.run(input_data_for_inference.cpu().numpy())[0]

print(f"\n\nONNXRuntime [INT8] Output length: {len(output_quan_oxrun)}")
print(f"ONNXRuntime [INT8] Sample output: {output_quan_oxrun}")
print(f"ONNXRuntime [INT8] Sample output type: {output_quan_oxrun.dtype}")
print(f"Estimate inference time:", (1/tensor_fps_int8)*1000 , "ms")

*************** EP Error ***************
EP Error /onnxruntime_src/onnxruntime/python/onnxruntime_pybind_state.cc:490 void onnxruntime::python::RegisterTensorRTPluginsAsCustomOps(PySessionOptions&, const onnxruntime::ProviderOptions&) Please install TensorRT libraries as mentioned in the GPU requirements page, make sure they're in the PATH or LD_LIBRARY_PATH, and that your GPU is supported.
 when using ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
Falling back to ['CUDAExecutionProvider', 'CPUExecutionProvider'] and retrying.
****************************************
Tensor FPS [INT8]: 146.69
Speedup: 1.26x


ONNXRuntime [INT8] Output length: 1
ONNXRuntime [INT8] Sample output: [[ -5.069152     0.64424354   1.4633828   -0.20066515  -8.01205
   -1.8507159   -8.890077    -0.07903468  -9.441972    -4.9386864
   -5.5257626    2.8989656   -6.1320357    5.6423135   -6.366642
  -12.955916     0.3661659   -8.492876    -0.7432854    4.5376787
   -1.2352638   -5.9

In [ ]:
val_with_img(onnx_engine_int8_pdqt)

Accuracy : 95.983936 %


In [ ]:
from onnxruntime.quantization import CalibrationDataReader, QuantType
import torch
class DataLoaderCalibrationReader(CalibrationDataReader):
    def __init__(self, dataloader, input_name):

        self.dataloader = dataloader
        self.input_name = input_name
        self.data_iter = iter(dataloader)

    def get_next(self):
        try:
            data = next(self.data_iter)
            inputs = data[0]  # Assuming input data is the first element in each batch
            # Convert inputs to numpy (if it's a PyTorch Tensor)
            if isinstance(inputs, torch.Tensor):
                inputs = inputs.numpy()
            # Return a dictionary where the key is the input name and value is the input data
            return {self.input_name: inputs}
        except StopIteration:
            return None

In [ ]:
# @title Optimized and Compressed (Quantization) to support INT8
from onnxruntime.quantization import quantize_static, QuantType

Dl_labeled = DataLoader(labeled_dataset, 1, shuffle=True, num_workers=2)
reader = DataLoaderCalibrationReader(Dl_labeled, 'input.1')

model_int8 = quantize_static(
    '/content/cp/onnx_fp32.onnx',
    '/content/cp/onnx_psqt_int8.onnx',
    weight_type=QuantType.QInt8,
    calibration_data_reader = reader  # You can use QuantType.QUInt8 for unsigned INT8
)
#---------------------------------------------------------


# Create a engine from the ONNX model and measure inference speed
model_onnx_int8_psqt = onnx.load('/content/cp/onnx_psqt_int8.onnx')
onnx_engine_int8_psqt = backend.prepare(model_onnx_int8_psqt, provider='CPUExecutionProvider', float16=True)

# Run a calibration step to collect statistics
num_iterations = 10000
total_time_int8 = 0.0
with torch.no_grad():
    for i in range(num_iterations):
        input_data = torch.randn(input_shape, dtype=torch.float32).cpu()
        start_time = time.time()
        output_data = onnx_engine_int8_psqt.run(input_data.cpu().numpy())[0]
        end_time = time.time()
        total_time_int8 += end_time - start_time
tensor_fps_int8 = num_iterations /total_time_int8
#tensor_fps = num_iterations / total_time

print(f"Tensor FPS [INT8]: {tensor_fps_int8:.2f}")
print(f"Speedup: {tensor_fps_int8/pytorch_fps:.2f}x")

*************** EP Error ***************
EP Error /onnxruntime_src/onnxruntime/python/onnxruntime_pybind_state.cc:490 void onnxruntime::python::RegisterTensorRTPluginsAsCustomOps(PySessionOptions&, const onnxruntime::ProviderOptions&) Please install TensorRT libraries as mentioned in the GPU requirements page, make sure they're in the PATH or LD_LIBRARY_PATH, and that your GPU is supported.
 when using ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
Falling back to ['CUDAExecutionProvider', 'CPUExecutionProvider'] and retrying.
****************************************
Tensor FPS [INT8]: 249.34
Speedup: 2.15x


In [ ]:
#---------Inference with INT8--------------
import onnxruntime as ort
input_data_for_inference = torch.randn(input_shape)
output_quan_oxrun = onnx_engine_int8_psqt.run(input_data_for_inference.cpu().numpy().astype(np.float32))[0]

print(f"\n\nONNXRuntime [INT8] Output length: {len(output_quan_oxrun)}")
print(f"ONNXRuntime [INT8] Sample output: {output_quan_oxrun}")
print(f"ONNXRuntime [INT8] Sample output type: {output_quan_oxrun.dtype}")
print(f"Estimate inference time:", (1/tensor_fps_int8)*1000 , "ms")



ONNXRuntime [INT8] Output length: 1
ONNXRuntime [INT8] Sample output: [[ -7.3484564  -1.0135802 -12.162962    6.588271  -10.389197   -1.2669753
   -7.8552465  -3.5475307  -4.814506    5.5746913 -16.470678   -3.0407405
   -7.6018515  -1.5203702  -5.5746913 -14.696913   -8.362037    2.5339506
    8.108642    2.2805555   1.5203702  -7.6018515 -12.416357   -2.2805555
    2.5339506   5.8280864   6.588271  -13.176542  -13.683332   -3.5475307
   12.669752 ]]
ONNXRuntime [INT8] Sample output type: float32
Estimate inference time: 4.010584497451783 ms


In [ ]:
val_with_img(onnx_engine_int8_psqt)

Accuracy : 93.775100 %


In [ ]:
import onnx
def count_parameters(model_path):
    model = onnx.load(model_path)
    total_params = 0

    for initializer in model.graph.initializer:
        param_array = onnx.numpy_helper.to_array(initializer)
        param_size = param_array.nbytes  # Size in bytes
        total_params += param_size

    return total_params

In [ ]:
MEGA = 1_000_000

model_fp32_path = "/content/cp/onnx_fp32.onnx" # 540kB
model_fp16_path = "/content/cp/onnx_fp16.onnx" # 296kB
model_pdqt_path = "/content/cp/onnx_pdqt_int8.onnx" # 290kB
model_psqt_path = "/content/cp/onnx_psqt_int8.onnx" # 367kB


In [ ]:
print(f'Number of model_fp32 params: {count_parameters(model_fp32_path)/MEGA} MB')
print(f'Number of model_fp16 params: {count_parameters(model_fp16_path)/MEGA} MB')
print(f'Number of model_pdqt params: {count_parameters(model_pdqt_path)/MEGA} MB')
print(f'Number of model_psqt params: {count_parameters(model_psqt_path)/MEGA} MB')

Number of model_fp32 params: 0.512124 MB
Number of model_fp16 params: 0.256062 MB
Number of model_pdqt params: 0.150306 MB
Number of model_psqt params: 0.146282 MB
